In [1]:
import sys
import glob
import numpy.matlib
import numpy as np
import cv2
import re
from termcolor import colored
from PIL import Image
from typing import List
from random import choices
import argparse
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
import torchvision
#check init out!
import torch.nn.init as init
import math

#import torchvision  #for deformconv2

import random
import configparser
from typing import Tuple
import matplotlib.pyplot as plt

from guided_filter_pytorch.guided_filter import GuidedFilter

In [2]:
#KITTI, MB or ETH
dataset = 'MB'
#used as prefix for saved weights
model_name = 'mb_60f40s'

#folder with training data
#input_folder = '/media/HDD/TrainingsData/kitti2012/training/'

#input_folder = '/media/HDD/TrainingsData/kitti2015/training/'

#input_folder = '/media/HDD/TrainingsData/MB_H/trainingHDispEL/*/'

#input_folder = '/media/HDD/Self-Sup_Stereo/OwnStereoNW/Out/Train/post-proc/NewTrainset/trainingHDisp/*/'

#input_folder = '/media/HDD/TrainingsData/MB_H/trainingHDispRef/*/'

input_folder = '/media/HDD/Self-Sup-Github/SAda-Net/TrainingRef/*'

save_folder_branch = '/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branch/'
save_folder_simb = '/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/simB/'


save_folder_branchdisp = '/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branchDisp/'
save_folder_branchim = '/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branchIm/'
save_folder_refine = '/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/refClass/'
w_reg_folder = '/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/refReg/'



out_folder = '/media/HDD/Self_supervised_BCKUP/Out/'


lr = 0.00006

batch_size = 100

num_feat_branch = 60 #46
num_feat_simb = 50 #45 

feat = 100

featref = 45

save_weights = 100000

#needs to be odd
#size of patch-crops fed into the network
#patch_size = 21
patch_size = 37#27

ps_h = int(patch_size/2)

#range for offset of o_neg
r_low = 1
r_high = 25

#inference for refiner network
upsampling_factor = 1
disp_scale = 1
downsampling_factor = 1
max_disp = 256


In [3]:
Tensor = torch.cuda.FloatTensor
LongTensor = torch.cuda.LongTensor
cos = torch.nn.CosineSimilarity()

In [4]:
class Sine(nn.Module):
    def __init__(self, w0: float = 1.0):
        super().__init__()
        self.w0 = w0

    def forward(self, x: torch.Tensor):
        return torch.sin(self.w0 * x)

In [5]:
class SiameseBranch64(nn.Module):
    def __init__(self,img_ch=3):
        super(SiameseBranch64,self).__init__()

        self.Tanh = nn.Tanh() 
        self.Conv1 = nn.Conv2d(img_ch, num_feat_branch, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)      
        self.Conv2 = nn.Conv2d(num_feat_branch, num_feat_branch, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv3 = nn.Conv2d(2*num_feat_branch, num_feat_branch, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv4 = nn.Conv2d(3*num_feat_branch, 60, kernel_size = 3,stride=1,padding = 1,dilation = 1,bias=True)  


    def forward(self,x_in):

        x1 = self.Conv1(x_in) 
        x1 = self.Tanh(x1)

        x2 = self.Conv2(x1) 
        x2 = self.Tanh(x2)

        d2 = torch.cat((x1,x2),dim=1)

        x3 = self.Conv3(d2) 
        x3 = self.Tanh(x3)

        d3 = torch.cat((x1,x2,x3),dim=1)

        x4 = self.Conv4(d3)

        return x4

branch = SiameseBranch64()
branch = branch.cuda()

#no end-to-end training right now!
branch.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/branch/mb_60f40s_best3800e3277684.000000'))


<All keys matched successfully>

In [6]:
class SimMeasTanh(nn.Module):
    def __init__(self,img_ch=2*60):
        super(SimMeasTanh,self).__init__()

        self.tanh = nn.Tanh() 

        self.Conv1 = nn.Conv2d(img_ch, num_feat_simb, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv2 = nn.Conv2d(num_feat_simb, num_feat_simb, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv3 = nn.Conv2d(2*num_feat_simb, num_feat_simb, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv4 = nn.Conv2d(3*num_feat_simb, num_feat_simb, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv5 = nn.Conv2d(4*num_feat_simb, 1, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)

    def forward(self,x_in):

        x1 = self.Conv1(x_in) 
        x1 = self.tanh(x1)

        x2 = self.Conv2(x1) 
        x2 = self.tanh(x2)

        d1 = torch.cat((x1,x2),dim=1)


        x3 = self.Conv3(d1) 
        x3 = self.tanh(x3)

        d2 = torch.cat((x1,x2,x3),dim=1)

        x4 = self.Conv4(d2) 
        x4 = self.tanh(x4) 
        d3 = torch.cat((x1,x2,x3,x4),dim=1)
        x5 = self.Conv5(d3)

        return x5


simB = SimMeasTanh()
simB = simB.cuda()

#no end-to-end training right now!
simB.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/simB/mb_60f40s_best3800e3277684.000000'))


<All keys matched successfully>

In [7]:
class BranchRefIm(nn.Module):
    def __init__(self,img_ch=3):
        super(BranchRefIm,self).__init__()

        self.Tanh = nn.Tanh() 
        
        self.Sine = Sine()
        self.Conv1 = nn.Conv2d(img_ch, feat, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)      
        self.Conv2 = nn.Conv2d(feat, feat, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv3 = nn.Conv2d(feat, feat, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv4 = nn.Conv2d(feat, 60, kernel_size = 3,stride=1,padding = 1,dilation = 1,bias=True)  


    def forward(self,x_in):

        x1 = self.Conv1(x_in) 
        #x1 = self.Tanh(x1)
        x1 = self.Sine(x1)

        x2 = self.Conv2(x1) 
        #x2 = self.Tanh(x2)
        x2 = self.Sine(x2)

        x3 = self.Conv3(x2) 
        #x3 = self.Tanh(x3)
        x3 = self.Sine(x3)

        x4 = self.Conv4(x3)

        return x4

branchIm = BranchRefIm()
branchIm = branchIm.cuda()




In [8]:
class BranchRefDisp(nn.Module):
    def __init__(self,img_ch=1):
        super(BranchRefDisp,self).__init__()

        self.Tanh = nn.Tanh() 
        self.Sine = Sine()
        self.Conv1 = nn.Conv2d(img_ch, feat, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)      
        self.Conv2 = nn.Conv2d(feat, feat, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv3 = nn.Conv2d(feat, feat, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv4 = nn.Conv2d(feat, 60, kernel_size = 3,stride=1,padding = 1,dilation = 1,bias=True)  


    def forward(self,x_in):

        x1 = self.Conv1(x_in) 
        #x1 = self.Tanh(x1)
        x1 = self.Sine(x1)

        x2 = self.Conv2(x1) 
        #x2 = self.Tanh(x2)
        x2 = self.Sine(x2)

        x3 = self.Conv3(x2) 
        #x3 = self.Tanh(x3)
        x3 = self.Sine(x3)

        x4 = self.Conv4(x3)

        return x4

branchDisp = BranchRefDisp()
branchDisp = branchDisp.cuda()



In [9]:
class RefClass(nn.Module):
    def __init__(self,img_ch=120):
        super(RefClass,self).__init__()

        self.Tanh = nn.Tanh() 
        self.Sine = Sine()
        
        self.Conv1 = nn.Conv2d(img_ch, featref, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)      
        self.Conv2 = nn.Conv2d(featref, featref, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv3 = nn.Conv2d(featref, featref, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        
        self.Conv4 = nn.Conv2d(featref, 380, kernel_size = 3,stride=1,padding = 1,dilation = 1,bias=True)  
        #self.softmax = nn.Softmax(dim=1)


    def forward(self,x_in):

        x1 = self.Conv1(x_in) 
        #x1 = self.Tanh(x1)
        x1 = self.Sine(x1)

        x2 = self.Conv2(x1) 
        #x2 = self.Tanh(x2)
        x2 = self.Sine(x2)
        
        x3 = self.Conv3(x2) 
        #x3 = self.Tanh(x3)
        x3 = self.Sine(x3)

        x4 = self.Conv4(x3)
        #x5 = self.softmax(x4)

        return x4

refClass = RefClass()
refClass = refClass.cuda()


In [10]:
featrefreg = 100
class RefReg(nn.Module):
    def __init__(self,img_ch=1):
        super(RefReg,self).__init__()

        self.Tanh = nn.Tanh() 
        self.Sine = Sine()
        
        self.Conv1 = nn.Conv1d(img_ch, featrefreg, kernel_size = 1,stride=1,padding = 0,dilation = 1, bias=True)      
        self.Conv2 = nn.Conv1d(featrefreg, featrefreg, kernel_size = 1,stride=1,padding = 0,dilation = 1, bias=True)
        self.Conv3 = nn.Conv1d(featrefreg, featrefreg, kernel_size = 1,stride=1,padding = 0,dilation = 1, bias=True)
        
        self.Conv4 = nn.Conv1d(featrefreg, 1, kernel_size = 1,stride=1,padding = 0,dilation = 1,bias=True)  


    def forward(self,x_in):

        x1 = self.Conv1(x_in) 
        #x1 = self.Tanh(x1)
        x1 = self.Sine(x1)

        x2 = self.Conv2(x1) 
        #x2 = self.Tanh(x2)
        x2 = self.Sine(x2)
        
        x3 = self.Conv3(x2) 
        #x3 = self.Tanh(x3)
        x3 = self.Sine(x3)

        x4 = self.Conv4(x3)
        #x5 = self.softmax(x4)
        x4 = self.Tanh(x4)
        
        return x4

refreg = RefReg()
refreg = refreg.cuda()


In [11]:

pytorch_branch_params = sum(p.numel() for p in branch.parameters() if p.requires_grad)
print("Nr feat branch: " ,pytorch_branch_params)


Nr feat branch:  196260


In [12]:

pytorch_simb_params = sum(p.numel() for p in simB.parameters() if p.requires_grad)
print("Nr feat simB: " ,pytorch_simb_params)


Nr feat simB:  191001


In [13]:

pytorch_im_params = sum(p.numel() for p in branchIm.parameters() if p.requires_grad)
print("Nr feat Im: " ,pytorch_im_params)


Nr feat Im:  237060


In [14]:

pytorch_disp_params = sum(p.numel() for p in branchDisp.parameters() if p.requires_grad)
print("Nr feat Disp: " ,pytorch_disp_params)


Nr feat Disp:  235260


In [15]:

pytorch_refine_params = sum(p.numel() for p in refClass.parameters() if p.requires_grad)
print("Nr feat Refine: " ,pytorch_refine_params)


Nr feat Refine:  239465


In [16]:

pytorch_refine_params = sum(p.numel() for p in refreg.parameters() if p.requires_grad)
print("Nr feat Regression: " ,pytorch_refine_params)


Nr feat Regression:  20501


In [17]:
def readPFM(file):
    file = open(file, 'rb')

    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().decode('utf-8').rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('utf-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().decode('utf-8').rstrip())
    if scale < 0:  # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>'  # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale

In [18]:
def loadMB(input_folder):
    
    left_filelist = glob.glob(input_folder + '/im0.png')
    gt_filelist = glob.glob(input_folder + '/disp0GT.pfm')
    calib_filelist = glob.glob(input_folder + '/calib.txt')
    pred_filelist = glob.glob(input_folder + '/dispPred.pfm')
    
    right_filelist = glob.glob(input_folder + '/im1.png')
    #gt_right_filelist = glob.glob(input_folder + '/disp1GT.pfm')
    #pred_right_filelist = glob.glob(input_folder + '/dispPredR.pfm')
    
    left_filelist = sorted(left_filelist)
    right_filelist = sorted(right_filelist)
    gt_filelist = sorted(gt_filelist)
    calib_filelist = sorted(calib_filelist)
    pred_filelist = sorted(pred_filelist)
    
    right_filelist = sorted(right_filelist)
   # gt_right_filelist = sorted(gt_right_filelist)
    #pred_right_filelist = sorted(pred_right_filelist)
    
    
    left_list = []
    gt_list = []
    maxdisp_list = []
    s_name_list = []
    pred_list = []
    
    right_list = []
    #predR_list = []
    #gtR_list = []
    
    for i in range(0,len(left_filelist)):
        
        cur_left = cv2.imread(left_filelist[i])
        cur_gt,_ = readPFM(gt_filelist[i])
        
        cur_gt[np.isnan(cur_gt)] = 0
        cur_gt[np.isinf(cur_gt)] = 0
        
        cur_pred, _ = readPFM(pred_filelist[i])
        pred_list.append(cur_pred)        
        
        cur_right = cv2.imread(right_filelist[i])        
        #cur_gtR, _ = readPFM(gt_right_filelist[i])
        #predR, _ = readPFM(pred_right_filelist[i])
        
       # cur_gtR[np.isnan(cur_gtR)] = 0
        #cur_gtR[np.isinf(cur_gtR)] = 0
        
        left_list.append(cur_left)
        gt_list.append(cur_gt)
        
        #just one list? should work in theory => try it!
       # left_list.append(cur_right)
        #just bc lazy
        right_list.append(cur_right)
        
        #gt_list.append(cur_gtR)
       # pred_list.append(predR)
        
        s_name = left_filelist[i].split('/')[-2]
       # s_name_list.append(s_name)
        s_name_list.append(s_name)
        
        f = open(calib_filelist[i],'r')
        calib = f.read()
        max_disp = int(calib.split('\n')[6].split("=")[1])
        
        
        maxdisp_list.append(max_disp)
                
        
    return left_list, right_list, gt_list, maxdisp_list, s_name_list, pred_list

In [19]:

left_list, right_list, gt_list,max_disp_list, s_name_list, pred_list = loadMB(input_folder)


In [20]:

cross_entropy = nn.CrossEntropyLoss()


In [21]:
def getBatch(): #gt_cons_cpy
    
    batch_xl = np.zeros((batch_size,3,patch_size,patch_size))
    
    batch_gt = np.zeros((batch_size,1, patch_size,patch_size)) 
    
    label_gt = np.zeros((batch_size,380, patch_size,patch_size))
    label_gt_int = np.zeros((batch_size,1, patch_size,patch_size))
    
    batch_pred = np.zeros((batch_size,1, patch_size,patch_size))
    
    
    for el in range(batch_size):
        
        if(el % 10 == 0):
            
            ridx = np.random.randint(0,len(left_list),1)
            left_im = left_list[ridx[0]]
            gt_im = gt_list[ridx[0]]
            pred_im = pred_list[ridx[0]]
        
        
        h,w,c = left_im.shape
        r_h = 0
        r_w = 0
        d = 0
        
#        print('Draw for random position')
        #also check height! should not draw corner pixels!!
        while True:
            
            r_h = random.sample(range(ps_h,h-(ps_h+1)), 1)
            r_w = random.sample(range(ps_h,w-(ps_h+1)),1)   
            
            if(not np.isnan(gt_im[r_h,r_w])):
                d = int(np.round(gt_im[r_h,r_w]))
                if((r_w[0]-ps_h-d-1) >= 0):
                     if((r_w[0]+(ps_h+1)-d+1) <= w):
                        break
        
        d = int(np.round(gt_im[r_h,r_w]))
                
        cur_left = left_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1),:]
        
        cur_gt = gt_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1)]
        cur_gt[np.isnan(cur_gt)] = 0.0
        cur_gt = np.expand_dims(cur_gt,axis=2)
        
        cur_pred = pred_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1)]
        cur_pred = np.expand_dims(cur_pred,axis=2)
        
        h,w,c = cur_gt.shape
        
        for h_ in range(h):
            for w_ in range(w):
                
                #print(cur_gtT[h_,w_])
                enc_pos = int(cur_gt[h_,w_,:])
                label_gt[el,enc_pos,h_,w_] = 1
                
                label_gt_int[el,:,h_,w_] = enc_pos
               
            
        #cur_gtT = torch.tensor(cur_gt)
        
        batch_xl[el,:,:,:] =  np.transpose(cur_left, (2,0,1)).astype(np.uint8)
        batch_gt[el,:,:,:] =  np.transpose(cur_gt, (2,0,1)).astype(np.uint8)
        
        batch_pred[el,:,:,:] =  np.transpose(cur_pred, (2,0,1)).astype(np.uint8)
        
        
    return batch_xl, batch_gt, batch_pred, label_gt, label_gt_int

In [22]:
def calcEPE(disp, gt_fn):
    
    gt = gt_fn

    gt[np.where(gt == np.inf)] = -100
    #for loadmb
    gt[np.where(gt == 0)] = -100
    
    mask = gt > 0

    disp = disp[mask]
    gt = gt[mask]        

    nr_px = len(gt)


    abs_error_im = np.abs(disp - gt)

    five_pe = (float(np.count_nonzero(abs_error_im >= 5.0) ) / nr_px) * 100.0  
    four_pe = (float(np.count_nonzero(abs_error_im >= 4.0) ) / nr_px) * 100.0  
    three_pe = (float(np.count_nonzero(abs_error_im >= 3.0) ) / nr_px) * 100.0  
    two_pe = (float(np.count_nonzero(abs_error_im >= 2.0) ) / nr_px) * 100.0        
    one_pe = (float(np.count_nonzero(abs_error_im >= 1.0) ) / nr_px) * 100.0        
    pf_pe = (float(np.count_nonzero(abs_error_im >= 0.5) ) / nr_px) * 100.0  
        
    return five_pe, four_pe, three_pe, two_pe, one_pe, pf_pe

In [23]:
def writePFM(file, image, scale=1):
    file = open(file, 'wb')

    color = None

    image = np.flipud(image)

    if len(image.shape) == 3 and image.shape[2] == 3:  # color image
        color = True
    elif len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1:  # greyscale
        color = False
    else:
        raise Exception('Image must have H x W x 3, H x W x 1 or H x W dimensions.')

    file.write('PF\n'.encode() if color else 'Pf\n'.encode())
    file.write('%d %d\n'.encode() % (image.shape[1], image.shape[0]))

    endian = image.dtype.byteorder

    scale = -scale

    file.write('%f\n'.encode() % scale)
    image.tofile(file)

In [24]:
def Eval(epoch):
    
    avg_4_pe = 0.0
    avg_2_pe = 0.0
    avg_1_pe = 0.0
    avg_pf_pe = 0.0

    for sample in range(len(left_list)):
        
        left = left_list[sample]
        pred = pred_list[sample]

        left = np.transpose(left, (2,0,1)).astype(np.uint8)
        
        with torch.no_grad():

            xlT = Variable(Tensor(left)).unsqueeze(0)
            predT = Variable(Tensor(pred.copy())).unsqueeze(0).unsqueeze(0)
            
            #xlT = (xlT-torch.mean(xlT))/torch.std(xlT)
            #predT = (predT-torch.mean(predT))/torch.std(predT)

            outXlT = branchIm(xlT)
            outDispT = branchDisp(predT)

            xlDispConcat = torch.cat((outXlT,outDispT),dim=1).squeeze(2)

            refDispT = refClass(xlDispConcat)

            pred = torch.argmax(refDispT, axis = 1)

            five_pe, four_pe, three_pe, two_pe, one_pe, pf_pe = calcEPE(pred.squeeze().cpu().data.numpy(), gt_list[sample])

            #print("2-PE: {}".format(two_pe))
            avg_4_pe = avg_4_pe + four_pe
            avg_2_pe = avg_2_pe + two_pe
            avg_1_pe = avg_1_pe + one_pe
            avg_pf_pe = avg_pf_pe + pf_pe

            writePFM('/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/%s_%06d_e%06f.pfm' %(s_name_list[sample], epoch,two_pe), pred.squeeze().cpu().data.numpy().astype(np.float32))

        
    avg_4_pe = avg_4_pe / len(left_list)
    avg_2_pe = avg_2_pe / len(left_list)
    avg_1_pe = avg_1_pe / len(left_list)
    avg_pf_pe = avg_pf_pe / len(left_list)
    
    print("Avg. 4-PE: {}".format(avg_4_pe))
    print("Avg. 2-PE: {}".format(avg_2_pe))
    print("Avg. 1-PE: {}".format(avg_1_pe))
    print("Avg. 0.5-PE: {}".format(avg_pf_pe))
    
    return avg_2_pe


In [26]:
#even further improve this by using pytorch!
def LR_Check(first_output, second_output, dataset):    
    
    h,w = first_output.shape
        
    line = np.array(range(0, w))
    idx_arr = np.matlib.repmat(line,h,1)    
    
    dif = idx_arr - first_output
    
    first_output[np.where(dif <= 0)] = 0
    
    first_output = first_output.astype(int)
    second_output = second_output.astype(int)
    dif = dif.astype(int)
    
    second_arr_reordered = np.array(list(map(lambda x, y: y[x], dif, second_output)))
    
    dif_LR = np.abs(second_arr_reordered - first_output)
    first_output[np.where(dif_LR >= 1.1)] = 0
    
    if(dataset == 'MB'):
        first_output[np.where(first_output <= 15.0)] = 0
        
    
    first_output = first_output.astype(np.float32)
    first_output[np.where(first_output == 0.0)] = np.nan
    
        
    return first_output

In [27]:
%load_ext cython

In [28]:
%%capture
%%cython -a
import numpy as np
import cython
#@cython.boundscheck(False)
#@cython.nonecheck(False)
@cython.wraparound(False)
cpdef float[:, :] FillIncons(unsigned char[:, :] mask, float[:, :] disp):

    cpdef int curnan, curnanh, curnanw,curw, w, h ,left, right, above, under, r_above, l_above, r_under, l_under
    cpdef float fill  
    cpdef int max_search
    
    max_search = 30
    
    w = mask.shape[1]
    h = mask.shape[0] 
    
    #BG
    idc = np.argwhere(np.isnan(disp))    
    for curnan in range(len(idc)):
        
        curnanh = idc[curnan][0]
        curnanw = idc[curnan][1]        
        if(mask[curnanh,curnanw] == 0):
            
            #whole scanline is nan => disp is 0
            if(all(np.isnan(disp[curnanh,:]))):
                #hole line set to 0!
                disp[curnanh,:] = 0.0
                
            #all px to the left are NaN
            if(all(np.isnan(disp[curnanh,0:curnanw]))):
                #go to the right
                curw = curnanw
                fill = 0
                while(np.isnan(disp[curnanh,curw]) and mask[curnanh,curnanw] == 0):
                    curw = curw +1
                    fill = disp[curnanh,curw]
                disp[curnanh,curnanw] = fill
                
            #else go left
            else:
                curw = curnanw
                fill = 0
                while(np.isnan(disp[curnanh,curw]) and mask[curnanh,curnanw] == 0):
                    curw = curw -1
                    fill = disp[curnanh,curw]
                disp[curnanh,curnanw] = fill 
    
    #FG
    idcFG = np.argwhere(np.isnan(disp))
    for curnan in range(len(idcFG)):
        
        curnanh = idcFG[curnan][0]
        curnanw = idcFG[curnan][1]
      
        left = 0
        right = 0
        above = 0
        under = 0

        r_above = 0
        l_above = 0
        r_under = 0
        l_under = 0      
        
        
        if(curnanw == 0):
            left = 0
        else:
            left = int(disp[curnanh,curnanw-1])
            
        counter = 0                                    
        while(np.isnan(disp[curnanh,curnanw+counter])):
            counter = counter +1                       
            if((curnanw+counter) >= w or counter >= max_search):
                right = 0
                break
            right = int(disp[curnanh,curnanw+counter])
        
        counter = 0                                    
        while(np.isnan(disp[curnanh+counter,curnanw])):
            counter = counter +1                       
            if((curnanh+counter) >= h or counter >= max_search):
                above = 0
                break       
            above = int(disp[curnanh+counter,curnanw])
             
        if(curnanh == 0):
            under = 0
        else:
            under = int(disp[curnanh-1,curnanw])
        
        
        counter = 0                                    
        while(np.isnan(disp[curnanh+counter,curnanw+counter])):
            counter = counter +1
            if((curnanh+counter) >= h or counter >= max_search):
                r_above = 0
                break
            if((curnanw+counter) >= w):
                r_above = 0
                break                        
            r_above = int(disp[curnanh+counter,curnanw+counter])     
        
        if(curnanh == 0 or curnanw == 0):
            l_under = 0
        else:
            l_under = int(disp[curnanh-1,curnanw-1])
             
        
        counter = 0      
        while(np.isnan(disp[curnanh+counter,curnanw-counter])):
            counter = counter +1
            if((curnanh+counter) >= h):
                l_above = 0
                break
            if((curnanw-counter) <= 0 or counter >= max_search):
                l_above = 0
                break
            l_above = int(disp[curnanh+counter,curnanw-counter])

        if(curnanh == 0 or curnanw >= w-1):
            r_under = 0
        else:
            r_under = int(disp[curnanh-1,curnanw+1])
         
        fill = np.median([left,right,above,under,r_above,l_above,r_under,l_under])
        disp[curnanh,curnanw] = fill

    return disp

In [29]:
def EvalLR(epoch):
    
    avg_4_pe = 0.0
    avg_2_pe = 0.0
    avg_1_pe = 0.0
    avg_pf_pe = 0.0

    predR_list = glob.glob('/media/HDD/Self-Sup-Github/SAda-Net/TrainingRef/*/dispPredR.pfm')
    right_list = glob.glob('/media/HDD/Self-Sup-Github/SAda-Net/TrainingRef/*/im1.png')
    
    right_list = sorted(right_list)
    predR_list = sorted(predR_list)
    
    for sample in range(len(left_list)):
        
        left = left_list[sample]
        pred = pred_list[sample]

        left = np.transpose(left, (2,0,1)).astype(np.uint8)
        
        right_f = right_list[sample]
        right = cv2.imread(right_f)
        predR_f = predR_list[sample]
        
        right = np.transpose(right, (2,0,1)).astype(np.uint8)
        
        predR, _ = readPFM(predR_f)
        
        with torch.no_grad():

            xlT = Variable(Tensor(left)).unsqueeze(0)
            predT = Variable(Tensor(pred.copy())).unsqueeze(0).unsqueeze(0)
            
            #xlT = (xlT-torch.mean(xlT))/torch.std(xlT)
            #predT = (predT-torch.mean(predT))/torch.std(predT)

            outXlT = branchIm(xlT)
            outDispT = branchDisp(predT)

            xlDispConcat = torch.cat((outXlT,outDispT),dim=1).squeeze(2)

            refDispT = refClass(xlDispConcat)

            pred = torch.argmax(refDispT, axis = 1)
                        
            #five_pe, four_pe, three_pe, two_pe, one_pe, pf_pe = calcEPE(pred.squeeze().cpu().data.numpy(), gt_list[sample])

            #print("2-PE: {}".format(two_pe))
            #avg_4_pe = avg_4_pe + four_pe
            #avg_2_pe = avg_2_pe + two_pe
            #avg_1_pe = avg_1_pe + one_pe
            #avg_pf_pe = avg_pf_pe + pf_pe


            ####################R#####################
            xrT = Variable(Tensor(right)).unsqueeze(0)
            predRT = Variable(Tensor(predR.copy())).unsqueeze(0).unsqueeze(0)
            
            outXrT = branchIm(xrT)
            outDispRT = branchDisp(predRT)

            xrDispConcat = torch.cat((outXrT,outDispRT),dim=1).squeeze(2)

            refDispRT = refClass(xrDispConcat)

            predR = torch.argmax(refDispRT, axis = 1)
            
            
            ###########################
            
            disp_s = LR_Check(pred.squeeze().cpu().data.numpy(), predR.squeeze().cpu().data.numpy(), 'MB')

            disp_s_arr = np.array(disp_s)
            im_disp = Image.fromarray(disp_s_arr) 
            im_disp = np.dstack((im_disp, im_disp, im_disp)).astype(np.uint8)    

            h,w = disp_s.shape

            shifted = cv2.pyrMeanShiftFiltering(im_disp, 7, 7)

            gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 1,
                cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

            disp_filled  = FillIncons(thresh, disp_s_arr)
            
            disp_filled = np.array(disp_filled)      
            
            five_pe, four_pe, three_pe, two_pe, one_pe, pf_pe = calcEPE(disp_filled, gt_list[sample])

            #print("2-PE: {}".format(two_pe))
            avg_4_pe = avg_4_pe + four_pe
            avg_2_pe = avg_2_pe + two_pe
            avg_1_pe = avg_1_pe + one_pe
            avg_pf_pe = avg_pf_pe + pf_pe
            
            writePFM('/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/%s_%06d_e%06f_filled.pfm' %(s_name_list[sample], epoch,two_pe),disp_filled)
            writePFM('/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/%s_%06d_e%06f.pfm' %(s_name_list[sample], epoch,two_pe), pred.squeeze().cpu().data.numpy().astype(np.float32))
            writePFM('/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/%s_%06d_e%06fR.pfm' %(s_name_list[sample], epoch,two_pe), predR.squeeze().cpu().data.numpy().astype(np.float32))
            
            
    
    avg_4_pe = avg_4_pe / len(left_list)
    avg_2_pe = avg_2_pe / len(left_list)
    avg_1_pe = avg_1_pe / len(left_list)
    avg_pf_pe = avg_pf_pe / len(left_list)
    
    print("Avg. 4-PE: {}".format(avg_4_pe))
    print("Avg. 2-PE: {}".format(avg_2_pe))
    print("Avg. 1-PE: {}".format(avg_1_pe))
    print("Avg. 0.5-PE: {}".format(avg_pf_pe))
    
    return avg_2_pe


In [30]:
#branchIm.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branchDisp/mb_best'))
#branchDisp.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branchIm/mb_best'))
#refClass.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/refClass/mb_best'))


branchIm.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branchIm/mb_60f40s_best395000e7.798881'))
branchDisp.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/branchDisp/mb_60f40s_best395000e7.798881'))
refClass.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/refClass/mb_60f40s_best395000e7.798881'))


<All keys matched successfully>

In [33]:
#avg_2_pe = EvalLR(0)

In [ ]:

#two_pe = Eval(0)


In [ ]:
#EvalLR(0)

In [ ]:
def loss_own(gt_i, probs):
        
    gt = torch.exp(-0.5 * torch.abs(self.labels - gt_i) ** 2 / 2) / 2

    -(gt * torch.log(probs + 1e-10))

    ce = torch.mean(ce, dim=2)
    ce = torch.mean(ce, dim=0)
    classification_loss = torch.sum(ce)
    
    return classification_loss

In [ ]:
params = list(branchIm.parameters()) +  list(branchDisp.parameters()) + list(refClass.parameters())

optimizer_G = optim.Adam(params, lr)

#optimizer_G = optim.SGD(params, 0.1, momentum=0.9)
best_err = 10000000000

early_stopping_count = 0

i = 0

batch_xl, batch_gt, batch_pred, label_gt, label_gt_int = getBatch()
loss_list = []

avg_2_pe = 10000

while(True):
    
    #do not train now!
    break
    
    epoch_loss = 0.0
    optimizer_G.zero_grad()
    

    batch_xlT = Variable(Tensor(batch_xl))
    batch_predT = Variable(Tensor(batch_pred))
    label_gtT = Variable(LongTensor(label_gt_int))
    label_gtF = Variable(Tensor(label_gt_int))
    
    #batch_xlT = (batch_xlT-torch.mean(batch_xlT))/torch.std(batch_xlT)
    #batch_predT = (batch_predT-torch.mean(batch_predT))/torch.std(batch_predT)

    outXlT = branchIm(batch_xlT)
    outDispT = branchDisp(batch_predT)

    xlDispConcat = torch.cat((outXlT,outDispT),dim=1).squeeze(2)

    refDispT = refClass(xlDispConcat)

    pred = torch.argmax(refDispT, axis = 1)
    ce_loss = cross_entropy(refDispT, label_gtT.squeeze())
        
    #classification_loss = ce_loss * torch.normal(torch.mean(label_gtF), torch.std(label_gtF)).mean()
    
    #classification_loss = torch.sum(ce)
    
    ce_loss.backward()
    
    loss_list.append(ce_loss.cpu().data.numpy())
    
    
    #loss_list.append(ce_loss.cpu().data.numpy())
    #ce_loss.backward()
    optimizer_G.step()
    
    
    i = i + 1
    
    #if(i % 100 == 0):
    batch_xl, batch_gt, batch_pred, label_gt, label_gt_int = getBatch()
        
        
    if(i % 1000 == 0):
        
        #batch_xl, batch_gt, batch_pred, label_gt, label_gt_int = getBatch()
        
        print(ce_loss.cpu().data.numpy())
        
        two_pe = Eval(i)
        if(two_pe < avg_2_pe):
            print("Improved: {}".format(two_pe))
            avg_2_pe = two_pe
            torch.save(branchIm.state_dict(), save_folder_branchim + model_name + '_best%04i' %(i) + 'e%04f' %(avg_2_pe)) 
            torch.save(branchDisp.state_dict(), save_folder_branchdisp + model_name + '_best%04i' %(i) + 'e%04f' %(avg_2_pe)) 
            torch.save(refClass.state_dict(), save_folder_refine + model_name + '_best%04i' %(i) + 'e%04f' %(avg_2_pe)) 
            early_stopping_count = 0
            
        else:
            print("Did not improve")
            early_stopping_count = early_stopping_count + 1
        if(early_stopping_count >= 20):
            print("Stop Training")
            break
            
            


In [ ]:
def loadRefineDisp():
    
    disp_ref_list_f = glob.glob('/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/BestOwn/*.pfm')
    disp_ref_list_f = sorted(disp_ref_list_f)
    
    disp_ref_list = []
    
    for i in range(len(disp_ref_list_f)):
        
        cur_disp_ref, _ = readPFM(disp_ref_list_f[i])
        disp_ref_list.append(cur_disp_ref)
        
    
    return disp_ref_list

In [ ]:
def _compute_binary_kernel(window_size: Tuple[int, int]) -> torch.Tensor:
    r"""Creates a binary kernel to extract the patches. If the window size
    is HxW will create a (H*W)xHxW kernel.
    """
    window_range: int = window_size[0] * window_size[1]
    kernel: torch.Tensor = torch.zeros(window_range, window_range)
    for i in range(window_range):
        kernel[i, i] += 1.0
    return kernel.view(window_range, 1, window_size[0], window_size[1])


def _compute_zero_padding(kernel_size: Tuple[int, int]) -> Tuple[int, int]:
    r"""Utility function that computes zero padding tuple."""
    computed: Tuple[int, ...] = tuple([(k - 1) // 2 for k in kernel_size])
    return computed[0], computed[1]


class MedianBlur(nn.Module):
    r"""Blurs an image using the median filter.

    Args:
        kernel_size (Tuple[int, int]): the blurring kernel size.

    Returns:
        torch.Tensor: the blurred input tensor.

    Shape:
        - Input: :math:`(B, C, H, W)`
        - Output: :math:`(B, C, H, W)`

    Example:
        >>> input = torch.rand(2, 4, 5, 7)
        >>> blur = kornia.filters.MedianBlur((3, 3))
        >>> output = blur(input)  # 2x4x5x7
    """

    def __init__(self, kernel_size: Tuple[int, int]) -> None:
        super(MedianBlur, self).__init__()
        self.kernel: torch.Tensor = _compute_binary_kernel(kernel_size)
        self.padding: Tuple[int, int] = _compute_zero_padding(kernel_size)

    def forward(self, input: torch.Tensor):  # type: ignore
        if not torch.is_tensor(input):
            raise TypeError("Input type is not a torch.Tensor. Got {}"
                            .format(type(input)))
        if not len(input.shape) == 4:
            raise ValueError("Invalid input shape, we expect BxCxHxW. Got: {}"
                             .format(input.shape))
        # prepare kernel
        b, c, h, w = input.shape
        tmp_kernel: torch.Tensor = self.kernel.to(input.device).to(input.dtype)
        kernel: torch.Tensor = tmp_kernel.repeat(c, 1, 1, 1)

        # map the local window to single vector
        features: torch.Tensor = F.conv2d(
            input, kernel, padding=self.padding, stride=1, groups=c)
        features = features.view(b, c, -1, h, w)  # BxCx(K_h * K_w)xHxW

        # compute the median along the feature axis
        median: torch.Tensor = torch.median(features, dim=2)[0]
        return median

# functiona api
def median_blur(input: torch.Tensor,
                kernel_size: Tuple[int, int]) -> torch.Tensor:
    r"""Blurs an image using the median filter.

    See :class:`~kornia.filters.MedianBlur` for details.
    """
    return MedianBlur(kernel_size)(input)

In [ ]:
def EvalReg(epoch):
    
    avg_4_pe = 0.0
    avg_2_pe = 0.0
    avg_1_pe = 0.0
    avg_pf_pe = 0.0

    for sample in range(len(left_list)):
        
        pred = disp_ref_list[sample]

        
        with torch.no_grad():

            predFullT = Variable(Tensor(pred.copy()))
            h,w = predFullT.shape
            outFullT = refreg(predFullT.unsqueeze(0))
            #outFullT = torch.reshape(outFullT, (h,w))
            
            pred_out = pred + outFullT.squeeze().cpu().data.numpy()
            
            five_pe, four_pe, three_pe, two_pe, one_pe, pf_pe = calcEPE(pred_out, gt_list[sample])

            avg_4_pe = avg_4_pe + four_pe
            avg_2_pe = avg_2_pe + two_pe
            avg_1_pe = avg_1_pe + one_pe
            avg_pf_pe = avg_pf_pe + pf_pe

            #writePFM('/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/%s_%06d_e%06f.pfm' %(s_name_list[sample], epoch,two_pe), pred_out)

        
    avg_4_pe = avg_4_pe / len(left_list)
    avg_2_pe = avg_2_pe / len(left_list)
    avg_1_pe = avg_1_pe / len(left_list)
    avg_pf_pe = avg_pf_pe / len(left_list)
    
   # print("Avg. 4-PE: {}".format(avg_4_pe))
    #print("Avg. 2-PE: {}".format(avg_2_pe))
    #print("Avg. 1-PE: {}".format(avg_1_pe))
    print("Avg. 0.5-PE: {}".format(avg_pf_pe))
    
    #0.5 PE should improve most
    return avg_pf_pe


In [ ]:
def RegLoss(gt_batch, ds_batch, pred_batch):
    
    ds = gt_batch - ds_batch
    loss = torch.abs(pred_batch - ds)
    torch.where(loss < -1.0, loss, 0.0)
    torch.where(loss > 1.0, loss, 0.0)
    
    return loss
    

In [ ]:
batch_size_reg = 100

In [ ]:
def getBatchReg(): #gt_cons_cpy
    
    batch_xl = np.zeros((batch_size_reg,3,patch_size,patch_size))
    
    batch_gt = np.zeros((batch_size_reg,1,patch_size,patch_size))
    
    batch_pred = np.zeros((batch_size_reg,1,patch_size,patch_size))
    
    
    for el in range(batch_size_reg):
        
        if(el % 10 == 0):
            
            ridx = np.random.randint(0,len(left_list),1)
            left_im = left_list[ridx[0]]
            gt_im = gt_list[ridx[0]]
            pred_im = disp_ref_list[ridx[0]]
        
        
        h,w,c = left_im.shape
        r_h = 0
        r_w = 0
        d = 0
        
#        print('Draw for random position')
        #also check height! should not draw corner pixels!!
        while True:
            
            r_h = random.sample(range(ps_h,h-(ps_h+1)), 1)
            r_w = random.sample(range(ps_h,w-(ps_h+1)),1)   
            
            if(not np.isnan(gt_im[r_h,r_w])):
                d = int(np.round(gt_im[r_h,r_w]))
                if((r_w[0]-ps_h-d-1) >= 0):
                     if((r_w[0]+(ps_h+1)-d+1) <= w):
                        break
        
        
        cur_left = left_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1),:]
        
        cur_gt = gt_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1)]
        
        cur_pred = pred_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1)]

        batch_gt[el,:,:,:] =  cur_gt.astype(np.float32)
        
        batch_pred[el,:,:,:] =  cur_pred.astype(np.float32)
        
        
    return batch_gt, batch_pred


In [ ]:

disp_ref_list = loadRefineDisp()


In [ ]:
params = list(refreg.parameters())

optimizer_G = optim.Adam(params, lr)

best_err = 10000000000

early_stopping_count = 0

i = 0

batch_gt, batch_pred = getBatchReg()
loss_list = []

avg_2_pe = 10000

while(True):
    

    epoch_loss = 0.0
    optimizer_G.zero_grad()
    

    predT = Variable(Tensor(batch_pred))

    outT = refreg(predT)

    batch_gtT = Variable(Tensor(batch_gt))

    loss = RegLoss(batch_gtT, predT, outT).mean()
    
    
    loss.backward()
    
    loss_list.append(loss.cpu().data.numpy())
    
    
    #loss_list.append(ce_loss.cpu().data.numpy())
    #ce_loss.backward()
    optimizer_G.step()
    
    
    i = i + 1
    
    #if(i % 100 == 0):
    
    batch_gt, batch_pred = getBatchReg()
        
        
    if(i %  10000 == 0): #20000
        
        #batch_xl, batch_gt, batch_pred, label_gt, label_gt_int = getBatch()
        
        print('--------------------')
        print("Epoch: {}".format(i))
        print("CE-loss: {}".format(loss.cpu().data.numpy()))
        
        two_pe = EvalReg(i)
        if(two_pe < avg_2_pe):
            print("Improved: {}".format(two_pe))
            avg_2_pe = two_pe
            torch.save(refreg.state_dict(), w_reg_folder + model_name + '_best%04i' %(i) + 'e%04f' %(avg_2_pe)) 
            early_stopping_count = 0
            
        else:
            print("Did not improve")
            early_stopping_count = early_stopping_count + 1
        if(early_stopping_count >= 20):
            print("Stop Training")
            break
        print('--------------------')
            
            
            

In [ ]:

refreg.load_state_dict(torch.load('/media/HDD/Self-Sup-Github/SAda-Net/weights/Trained/refReg/mb_60f40s_best230000e24.911293'))

two_pe = EvalReg(i)


In [ ]:
folder = '/media/HDD/Self-Sup-Github/SAda-Net/OUTNEW/'

avg_4_pe = 0.0
avg_2_pe = 0.0
avg_1_pe = 0.0
avg_pf_pe = 0.0

for i in range(len(disp_ref_list)):

    dispT = Variable(Tensor(disp_ref_list[i].copy()))

    dispTfilt = median_blur(dispT.unsqueeze(0).unsqueeze(0), (5,5))

    filtered_disp = dispTfilt.squeeze().cpu().data.numpy()

    five_pe, four_pe, three_pe, two_pe, one_pe, pf_pe = calcEPE(filtered_disp, gt_list[i])
    
    avg_4_pe = avg_4_pe + four_pe
    avg_2_pe = avg_2_pe + two_pe
    avg_1_pe = avg_1_pe + one_pe
    avg_pf_pe = avg_pf_pe + pf_pe
    
    name = s_name_list[i]
    writePFM(folder + '%s_e%06f.pfm' %(s_name_list[i], two_pe), filtered_disp.astype(np.float32))
    
    
avg_4_pe = avg_4_pe / len(disp_ref_list)
avg_2_pe = avg_2_pe / len(disp_ref_list)
avg_1_pe = avg_1_pe / len(disp_ref_list)
avg_pf_pe = avg_pf_pe / len(disp_ref_list)

print("Avg. 4-PE {}".format(avg_4_pe))
print("Avg. 2-PE {}".format(avg_2_pe))
print("Avg. 1-PE {}".format(avg_1_pe))
print("Avg. 0.5-PE {}".format(avg_pf_pe))


